In [6]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import language_check
import re

In [17]:
def get_robots (url):
    response = requests.get(url+"/robots.txt")
    robots = response.text
    return robots

In [8]:
#COPY FROM ANDREA'S SCRAPER

In [9]:
def get_sentences(url, find, headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}):
    r = requests.get(url, headers=headers)
    # make sure that the page exist
    if r.status_code == 200:
        html = r.content
        try:
            dammit = UnicodeDammit(html)
            #print(dammit.unicode_markup)
            encoding = dammit.original_encoding
            soup = BeautifulSoup(html, 'lxml', exclude_encodings=[encoding])
            new_encoding = soup.original_encoding
            #print(soup.original_encoding)
            soup = BeautifulSoup(html, 'lxml',  exclude_encodings=[new_encoding])
            #print(soup.original_encoding)
            title = soup.find('h1')
            if title is not None:
                title_text = title.text.strip()
            #article = soup.find(find)   
            article = soup.find_all(find)
            new_article = []
            for i in article:
                dammit = UnicodeDammit(i.text.strip().encode(), smart_quotes_to="ascii").unicode_markup
                new_article.append((dammit.encode('ascii', 'ignore')).decode("utf-8"))
            sentences = []
            for i in new_article:
                sentences.append(sent_tokenize(i))
            sentences = sum(sentences, [])
            return sentences
        except TypeError:
            print("Type error with url: ", url)

def multi_sites(url_list, find):
    all_sents = []
    new_url = []
    for i in url_list:
        this_url_sents = get_sentences(i, find)
        if not isinstance(this_url_sents, type(None)):
            new_url.append(i)
            all_sents.append(this_url_sents)
    all_sents = sum(all_sents,[])
    return all_sents, new_url


In [11]:
def get_sentences_container(url, find, headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}):
    r = requests.get(url, headers=headers)
    # make sure that the page exist
    if r.status_code == 200:
        html = r.content
        try:
            dammit = UnicodeDammit(html)
            #print(dammit.unicode_markup)
            encoding = dammit.original_encoding
            soup = BeautifulSoup(html, 'lxml', exclude_encodings=[encoding])
            new_encoding = soup.original_encoding
            #print(soup.original_encoding)
            soup = BeautifulSoup(html, 'lxml',  exclude_encodings=[new_encoding])
            #print(soup.original_encoding)
            title = soup.find('h1')
            if title is not None:
                title_text = title.text.strip()
            #article = soup.find(find)   
            article = soup.find_all(find).text
            new_article = []
            for i in article:
                dammit = UnicodeDammit(i.text.strip().encode(), smart_quotes_to="ascii").unicode_markup
                new_article.append((dammit.encode('ascii', 'ignore')).decode("utf-8"))
            sentences = []
            for i in new_article:
                sentences.append(sent_tokenize(i))
            sentences = sum(sentences, [])
            return sentences
        except TypeError:
            print("Type error with url: ", url)
    
def multi_sites_container(url_list, find):
    all_sents = []
    new_url = []
    for i in url_list:
        this_url_sents = get_sentences_container(i, find)
        if not isinstance(this_url_sents, type(None)):
            new_url.append(i)
            all_sents.append(this_url_sents)
    all_sents = sum(all_sents,[])
    return all_sents, new_url

In [12]:
def save_csv(batch, file_name):
    batch = np.asarray(batch)
    pd_df = pd.DataFrame(batch)
    pd_df.to_csv(file_name, index = False, header = False)

In [13]:
def grammar_check(batch):
    grammar_tool = language_check.LanguageTool('en-GB')
    checked = []
    for i in batch:
        corrected = ''
        matches = grammar_tool.check(i)
        checked.append(language_check.correct(i, matches))
    return checked

In [20]:
first_url = 'http://www.dating-relationship-advice-for-women.com'
robots = get_robots(first_url)

In [21]:
print(robots)

#
User-agent: *
Disallow: /cgi-bin
Disallow: /wp-admin
Disallow: /wp-includes
Disallow: /wp-content/plugins/
Disallow: /wp-content/cache/
Disallow: /wp-content/themes/
Disallow: */trackback/
Disallow: */feed/
Disallow: /*/feed/rss/$
Disallow: /category/*


In [ ]:
#I'm guessing these are html folders.Trying to see how to verify in the articles url that anything is in those folders. 